## Key words
- cifar10
- vgg16 (freezing params)
- transfer learning

### packages and globe settings

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
from matplotlib import pyplot as plt

import torch
from torch import nn, optim
from torchvision import datasets, transforms, utils, models
from torch.utils import data
from torchkeras import summary, Model
from sklearn.metrics import precision_score, accuracy_score
import pandas as pd
import os
import datetime


CIFAR_ROOT = os.path.join('..', 'data')
CIFAR10_PATH = os.path.join(CIFAR_ROOT, 'cifar-10-batches-py')

HISTORY_FILE = os.path.join(CIFAR10_PATH, 'c2_cifar10_history.csv')
WEIGHT_FILE = os.path.join(CIFAR10_PATH, 'c2_cifar10_weight.pth')

HISTORY1_FILE = os.path.join(CIFAR10_PATH, 'c2_cifar10_history1.csv')
HISTORY2_FILE = os.path.join(CIFAR10_PATH, 'c2_cifar10_history2.csv')

WEIGHT1_FILE = os.path.join(CIFAR10_PATH, 'c2_cifar10_weight1.pth')
WEIGHT2_FILE = os.path.join(CIFAR10_PATH, 'c2_cifar10_weight2.pth')

NB_CLASSES = 10
NROWS = 8

IMAGE_MEAN = 0.5
IMAGE_STD = 0.5
IMAGE_SIZE = 32
IMAGE_CHANNEL = 3

BATCH_SIZE = 32
VAL_BATCH_SIZE = 64
LR = 1e-3

### common codes

In [2]:
# plot
def plot_metric(dfhistory, metric):
    train_metrics = dfhistory[metric]
    val_metrics = dfhistory['val_'+metric]
    epochs = range(1, len(train_metrics) + 1)
    plt.plot(epochs, train_metrics, 'bo--')
    plt.plot(epochs, val_metrics, 'ro-')
    plt.title('Training and validation '+ metric)
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend(["train_"+metric, 'valid_'+metric])
    plt.show()

def plot_images(features, mean=0.5, std=0.5, nrows=8, figsize=(2, 2)):
    # images: tensor (B, C, H, W), grid_image: ndarray (C, H, W)
    grid_image = utils.make_grid(features, nrow=nrows).numpy()
    grid_image = mean + grid_image * std

    # imshow (H, W, C)
    grid_image = grid_image.transpose(1, 2, 0)

    plt.figure(figsize=figsize)
    plt.imshow(grid_image)
    plt.xticks([])
    plt.yticks([])
    plt.show()

# save and load
def save_history(model, file, mode='csv'):
    assert mode == 'csv'
    assert type(model.history) is pd.DataFrame
    model.history.to_csv(file)

def save_weight(model, file):
    weights = dict()
    weights.update({'epoch': model.epoch})
    weights.update({'net': model.state_dict()})
    weights.update({'optimizer': model.optim.state_dict()})
    torch.save(weights, file)

def load_history(file, index_col='epoch', mode='csv'):
    assert mode == 'csv'
    return pd.read_csv(file, index_col=index_col)

def load_weight(model, file, net_only=False):
    weights = torch.load(file)
    model.load_state_dict(weights['net'])
    if not net_only:
        model.epoch = weights.get('epoch', 0)
        model.optim.load_state_dict(weights['optimizer'])
    return model

# metrics
def precision_metrics(targets, labels):
    # targets (-1, C), labels (-1)
    y_pred = targets.data.max(1)[1].numpy()
    y_true = labels.numpy()
    score = precision_score(y_true, y_pred, average='macro')
    # return (1)
    return torch.tensor(score)

def accuracy_metrics(targets, labels):
    # targets (-1, C), labels (-1)
    y_pred = targets.data.max(1)[1].numpy()
    y_true = labels.numpy()
    score = accuracy_score(y_true, y_pred)
    # return (1)
    return torch.tensor(score)

# training functions
def run_step(model, features, labels, train_mode=True):
    targets = model(features)
    
    metrics = dict()
    loss = model.loss_fn(targets, labels)
    metrics.update({'%sloss' % ('' if train_mode else 'val_'): loss.item()})
    
    for metric_name, metric_fn in model.metrics_dict.items():
        metric_value = metric_fn(targets, labels)
        metrics.update({'%s%s' % ('' if train_mode else 'val_', metric_name): metric_value.item()})

    loss.backward()
    model.optim.step()
    model.optim.zero_grad()

    return metrics

def run_epoch(model, dataloader, train_mode=True, log_per_steps=200):
    metrics_epoch = dict()

    model.train(train_mode)
    for step, (features, labels) in enumerate(dataloader, 1):
        metrics = run_step(model, features, labels, train_mode)

        # # update loss_epoch (mean)
        # loss_epoch = (step - 1) / step * loss_epoch + metric_val / step
        # update metric_epoch (mean)
        for metric_name, metric_val in metrics.items():
            if metrics_epoch.get(metric_name) == None:
                metrics_epoch[metric_name] = metric_val
            else:
                metrics_epoch[metric_name] = \
                    (step - 1) / step * metrics_epoch[metric_name] + metric_val / step

        if step % log_per_steps == 0:
            print(" - Step %d, %s" % (step, metrics_epoch))

    return metrics_epoch

def train_model(model, dataloader_train, dataloader_valid, epochs, log_per_epochs=10, log_per_steps=200):
    print("==========" * 6)
    print("= Training model")
    
    metrics_list = []
    start_epoch = 1 + model.epoch
    end_epoch = epochs + 1 + model.epoch
    for epoch in range(start_epoch, end_epoch):
        metrics = dict()
        print("==========" * 6)
        print("= Epoch %d/%d @ %s" % (epoch, end_epoch - 1, datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')))
        metrics_train = run_epoch(model, dataloader_train, train_mode=True, log_per_steps=log_per_steps)
        metrics_valid = run_epoch(model, dataloader_valid, train_mode=False, log_per_steps=log_per_steps)
        metrics.update({'epoch': epoch})
        metrics.update(metrics_train)
        metrics.update(metrics_valid)
        metrics_list.append(metrics)

        model.epoch = epoch

        if epoch % log_per_epochs == 0:
            print('= %s' % metrics)
        
    print("==========" * 6)
    
    model.history = pd.DataFrame(metrics_list)
    model.history.set_index('epoch', inplace=True)
    return model.history

def predict_model(model, features):
    model.eval()
    targets = model(features)
    
    return targets.data.max(1)[1]

def eval_model(model, features, labels):
    model.eval()
    targets = model(features)

    metrics = dict()
    for metric_name, metric_fn in model.metrics_dict.items():
        metric_value = metric_fn(targets, labels)
        metrics.update({metric_name: metric_value.item()})
    
    return metrics

### datasets and dataloader

In [3]:
# datasets and dataloader
data_tf = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomGrayscale(),
    transforms.ToTensor(),  # 0~255 -> 0~1
    transforms.Normalize(IMAGE_MEAN, IMAGE_STD) # 0~1 -> -1~1
])

ds_train = datasets.CIFAR10(CIFAR_ROOT, train=True, transform=data_tf, download=True)
ds_valid = datasets.CIFAR10(CIFAR_ROOT, train=False, transform=data_tf, download=True)

dl_train = data.DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True)
dl_valid = data.DataLoader(ds_valid, batch_size=VAL_BATCH_SIZE, shuffle=True)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


### batch sample plot (optional)

In [ ]:
# batch sample plot
batch_features, _ = next(iter(dl_train))

plot_images(batch_features, mean=IMAGE_MEAN, std=IMAGE_STD, nrows=NROWS, figsize=(8, 8))

### network class for transfer learning

In [6]:
class SimpleVGG16(nn.Module):
    def __init__(self, classes=10, *args, **kwargs):
        super(SimpleVGG16, self).__init__(*args, **kwargs)
        self.epoch = 0
        
        self.vgg16 = models.vgg16(pretrained=True)
        self._freeze_vgg16()
        
        self.fc1 = nn.Linear(1000, classes)
        self.logsoftmax1 = nn.LogSoftmax(1)

    def _freeze_vgg16(self):
        for param in self.vgg16.parameters():
            param.requires_grad = False

    def forward(self, input):
        input = self.vgg16(input)
        input = self.fc1(input)
        input = self.logsoftmax1(input)
        return input

In [7]:
Model(SimpleVGG16()).summary(input_shape=(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,792
              ReLU-2           [-1, 64, 32, 32]               0
            Conv2d-3           [-1, 64, 32, 32]          36,928
              ReLU-4           [-1, 64, 32, 32]               0
         MaxPool2d-5           [-1, 64, 16, 16]               0
            Conv2d-6          [-1, 128, 16, 16]          73,856
              ReLU-7          [-1, 128, 16, 16]               0
            Conv2d-8          [-1, 128, 16, 16]         147,584
              ReLU-9          [-1, 128, 16, 16]               0
        MaxPool2d-10            [-1, 128, 8, 8]               0
           Conv2d-11            [-1, 256, 8, 8]         295,168
             ReLU-12            [-1, 256, 8, 8]               0
           Conv2d-13            [-1, 256, 8, 8]         590,080
             ReLU-14            [-1, 25

### model training and save history and weights

In [8]:
# training settings (loss, optim & metrics)
model = SimpleVGG16(NB_CLASSES)
model.loss_fn = nn.CrossEntropyLoss()
model.optim = optim.Adam(model.parameters(), lr=LR)
model.metrics_dict = {
    'precision': precision_metrics,
    'accuracy': accuracy_metrics
}

# model training
history = train_model(model, dl_train, dl_valid, epochs=20, log_per_epochs=1, log_per_steps=200)

# save training history
save_history(model, HISTORY1_FILE)

# save weights
save_weight(model, WEIGHT1_FILE)

 400, {'loss': 1.691487196683885, 'precision': 0.47192662746204433, 'accuracy': 0.48578125000000033}
 - Step 600, {'loss': 1.705613253613314, 'precision': 0.47075569255847016, 'accuracy': 0.4846354166666669}
 - Step 800, {'loss': 1.7007703655213136, 'precision': 0.4707210082355916, 'accuracy': 0.4834765625000002}
 - Step 1000, {'loss': 1.6998088481426243, 'precision': 0.4717187454212451, 'accuracy': 0.4834687500000003}
 - Step 1200, {'loss': 1.6915812368690968, 'precision': 0.4744865594025311, 'accuracy': 0.48666666666666614}
 - Step 1400, {'loss': 1.681876653858596, 'precision': 0.47497808014296045, 'accuracy': 0.4872991071428568}
= {'epoch': 2, 'loss': 1.6812028925310543, 'precision': 0.4741547042485401, 'accuracy': 0.4867842290467045, 'val_loss': 1.2703908362965677, 'val_precision': 0.5625284020338351, 'val_accuracy': 0.560509554140127}
= Epoch 3/20 @ 2020-12-18 14:18:57
 - Step 200, {'loss': 1.6622813928127291, 'precision': 0.4742185966810964, 'accuracy': 0.4962499999999999}
 - Ste

### re-training

In [ ]:
model = SimpleVGG16(NB_CLASSES)
model.loss_fn = nn.CrossEntropyLoss()
model.optim = optim.Adam(model.parameters(), lr=LR)
model.metrics_dict = {
    'precision': precision_metrics,
    'accuracy': accuracy_metrics
}

# load weights
model = load_weight(model, WEIGHT1_FILE)
history = train_model(model, dl_train, dl_valid, 10, log_per_epochs=1, log_per_steps=200)

save_history(model, HISTORY2_FILE)
save_weight(model, WEIGHT2_FILE)
history

### metrics plot

In [ ]:
history = load_history(HISTORY_FILE)

In [ ]:
plot_metric(history, 'loss')

In [ ]:
plot_metric(history, 'precision')

### prediction

In [ ]:
features, _ = next(iter(dl_valid))

model = SimpleVGG16(NB_CLASSES)
model = load_weight(model, WEIGHT_FILE, net_only=True)

targets = predict_model(model, features)
print(targets.numpy().reshape(-1, NROWS))

plot_images(features, mean=IMAGE_MEAN, std=IMAGE_STD, nrows=NROWS)

### evaluation

In [ ]:
features, labels = next(iter(dl_valid))

model = SimpleVGG16(NB_CLASSES)
model.metrics_dict = {
    'precision': precision_metrics,
    'accuracy': accuracy_metrics
}
model = load_weight(model, WEIGHT_FILE, net_only=True)

metrics = eval_model(model, features, labels)

# print(labels.reshape(-1, NROWS))
# plot_images(features)
print(metrics)

In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
model = models.vgg16()
# Model(model).summary(input_shape=(3, 32, 32))
model.state_dict()['features.0.weight'][0, 0, :]

tensor([[-0.0046, -0.0142, -0.0822],
        [ 0.0523, -0.0474,  0.0054],
        [-0.0129, -0.0052, -0.0351]])

In [25]:
model1 = models.vgg16(pretrained=True)
model1.state_dict()['features.0.weight'][0, 0, :]

tensor([[-0.5537,  0.1427,  0.5290],
        [-0.5831,  0.3566,  0.7657],
        [-0.6902, -0.0480,  0.4841]])

In [26]:
model2 = SimpleVGG16()

In [31]:
model2 = load_weight(model2, WEIGHT1_FILE, net_only=True)

In [32]:
model2.state_dict()['vgg16.features.0.weight'][0, 0, :]

tensor([[-0.5784,  0.1180,  0.5037],
        [-0.6086,  0.3356,  0.7520],
        [-0.7151, -0.0606,  0.4800]])